In [1]:
import pandas as pd
import numpy as np
import glob
import warnings
warnings.filterwarnings("ignore")
import datetime
from datetime import datetime

# Air Pollution Data from six stations

In [2]:
poll = pd.DataFrame()
for file_name in glob.glob('data/airquality/'+'*.csv'):
    x = pd.read_csv(file_name, low_memory=False)
    x['Date']=pd.to_datetime(x['Date'])
    x['odate']=pd.to_datetime(file_name[-14:-4],format="%Y-%m-%d")
    poll = pd.concat([poll,x],axis=0)

In [3]:
poll.head(2)

,DeviceID,TSP,PM2.5,O3,CO,NO2,SO2,Temperature,Humid,Date,odate
0,0.0,24.9,11.1,0.0,2.40,0.000,0.37,39.9,39.6,2021-07-01 16:18:27,2021-07-01
1,0.0,29.5,11.6,0.0,1.12,0.044,0.21,40.0,39.8,2021-07-01 16:20:15,2021-07-01


In [4]:
poll=poll.dropna(subset=['Date'])
time=[]
for i in poll['Date']:
    time.append(i.strftime("%H:%M:%S"))
poll['date']=poll['odate'].astype(str)+" "+time
poll['date']=pd.to_datetime(poll['date'])

In [5]:
poll=poll[['DeviceID','date','TSP', 'PM2.5', 'O3', 'CO', 'NO2', 'SO2', 'Temperature', 'Humid']]
poll=poll.query('DeviceID>0 & DeviceID<7')
poll.reset_index(drop=True, inplace=True)
poll.head(3)

,DeviceID,date,TSP,PM2.5,O3,CO,NO2,SO2,Temperature,Humid
0,1.0,2021-02-20 11:44:11,15.4,9.2,13.0,1.32,0.015,0.14,26.1,60.5
1,1.0,2021-02-20 11:44:11,15.4,9.2,13.0,1.32,0.015,0.14,26.1,60.5
2,1.0,2021-02-20 11:44:11,15.4,9.2,13.0,1.32,0.015,0.14,26.1,60.5


,DeviceID,TSP,PM2.5,O3,CO,NO2,SO2,Temperature,Humid,Date,odate,date
0,0.0,24.9,11.1,0.0,2750.40,0.000,969.4,39.9,39.6,2021-07-01 16:18:27,2021-07-01,2021-07-01 16:18:27
1,0.0,29.5,11.6,0.0,1283.52,82.808,550.2,40.0,39.8,2021-07-01 16:20:15,2021-07-01,2021-07-01 16:20:15
2,0.0,35.5,11.9,0.0,1203.30,84.690,524.0,40.0,39.7,2021-07-01 16:21:15,2021-07-01,2021-07-01 16:21:15


In [6]:
poll_main = pd.DataFrame()
for i in range(1,7):
    s = poll.query('DeviceID==@i')
    temp=s.resample('60T',on='date').agg(np.nanmean).reset_index()
    poll_main = poll_main.append(temp)
poll_main['DeviceID']=poll_main['DeviceID'].ffill()
poll_main.reset_index(drop=True, inplace=True)

In [8]:
poll_main['CO']=poll_main['CO']*1146
poll_main['O3']=poll_main['O3']*1.963
poll_main['NO2']=poll_main['NO2']*1882
poll_main['SO2']=poll_main['SO2']*2620
poll_main.head(3)

,date,DeviceID,TSP,PM2.5,O3,CO,NO2,SO2,Temperature,Humid
0,2021-02-20 11:00:00,1.0,15.4,9.2,25.519,1512.72,28.23,366.8,26.1,60.5
1,2021-02-20 12:00:00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-02-20 13:00:00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
poll_main.loc[(poll_main['TSP']<0),'TSP']=np.nan
poll_main.loc[(poll_main['PM2.5']<0),'PM2.5']=np.nan
poll_main.loc[(poll_main['O3']<0),'O3']=np.nan
poll_main.loc[(poll_main['CO']<0),'CO']=np.nan
poll_main.loc[(poll_main['NO2']<0),'NO2']=np.nan
poll_main.loc[(poll_main['SO2']<0),'SO2']=np.nan

In [20]:
met_poll.rename({'DeviceID':'Station_No'}, axis=1, inplace=True)
met_poll.to_csv('data/AirQuality_hcmc.csv', index=False)